<a href="https://colab.research.google.com/github/LucianoRodriguez0764/ipadapter_faceid_colab/blob/main/Ipadapter_faceID_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependencies and download the IP-Adapter model

In [ ]:
!pip install insightface
!pip install onnxruntime
!pip install diffusers
!pip install git+https://github.com/tencent-ailab/IP-Adapter.git
!pip install einops

# Generate the image with IP-Adapter

In [2]:
!wget -O ip-adapter-faceid-plusv2_sd15.bin https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin

--2024-11-25 00:55:38--  https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin
Resolving huggingface.co (huggingface.co)... 18.239.50.16, 18.239.50.80, 18.239.50.49, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/36/ca/36ca54ff1895000df817b45a5ae71fccd14170b0843d8adc2ba7944d9ac903e9/26d0d86a1d60d6cc811d3b8862178b461e1eeb651e6fe2b72ba17aa95411e313?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27ip-adapter-faceid-plusv2_sd15.bin%3B+filename%3D%22ip-adapter-faceid-plusv2_sd15.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1732755338&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjc1NTMzOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzM2L2NhLzM2Y2E1NGZmMTg5NTAwMGRmODE3YjQ1YTVhZTcxZmNjZDE0MTcwYjA4NDNkOGFkYzJiYTc5NDRkOWFjOTAzZTkvMjZkM

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoencoderKL
from PIL import Image as PILImage

from ip_adapter.ip_adapter_faceid import IPAdapterFaceIDPlus

v2 = True
base_model_path = "SG161222/Realistic_Vision_V4.0_noVAE"
vae_model_path = "stabilityai/sd-vae-ft-mse"
image_encoder_path = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
ip_ckpt = "ip-adapter-faceid-plus_sd15.bin" if not v2 else "ip-adapter-faceid-plusv2_sd15.bin"
device = "cuda"

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
)



In [ ]:
import cv2
from insightface.app import FaceAnalysis
from insightface.utils import face_align
import torch
import glob, os
from PIL import Image as PILImage

app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

image_path = "images/embeds/1"
imgs = glob.glob(os.path.join(image_path, '*.*'))
#imgs += glob.glob(os.path.join('images/face_2', '*.*'))

# Collect all embeddings
embeddings = []

for i in range(len(imgs)):
    print(imgs[i])
    image = cv2.imread(imgs[i])
    faces = app.get(image)

    # Check if a face is detected
    if faces:
        faceid_embed = torch.from_numpy(faces[0].normed_embedding).unsqueeze(0)
        embeddings.append(faceid_embed)
    else:
        print("face not detected")

# Calculate the average embedding for the face ID

weights = []
weights = [elem/sum(weights) for elem in weights]
#print(weights)

if embeddings:

    ### weights
    total = torch.zeros_like(embeddings[0])
    for i in range(len(embeddings)):
        if len(weights)==len(embeddings):
            total.add_(embeddings[i]*weights[i])
        else:
            total.add_(embeddings[i]*(1/len(embeddings)))
    faceid_embeds = total / len(embeddings[i])

    ###  mean weights
    faceid_embeds_mean = torch.mean(torch.stack(embeddings), dim=0)

    print("Collected face ID embedding")
else:
    print("No faces detected in the images provided.")



# face
image_path_face = "images/face"
imgs_2 = glob.glob(os.path.join(image_path_face, '*.*'))
image1 = cv2.imread(imgs_2[0])
faces1 = app.get(image1)
if faces1:
  print("face image detected")
else:
  print("face image not detected")

face_image = face_align.norm_crop(image1, landmark=faces1[0].kps, image_size=336)

In [ ]:
# face
image_path_face = "images/face"
imgs_2 = glob.glob(os.path.join(image_path_face, '*.*'))
image1 = cv2.imread(imgs_2[3])
faces1 = app.get(image1)
if faces1:
  print("face image detected")
else:
  print("face image not detected")

face_image4 = face_align.norm_crop(image1, landmark=faces1[0].kps, image_size=336)

image1 = cv2.imread(imgs_2[2])
faces1 = app.get(image1)
if faces1:
  print("face image detected")
else:
  print("face image not detected")

face_image3 = face_align.norm_crop(image1, landmark=faces1[0].kps, image_size=336)

image1 = cv2.imread(imgs_2[1])
faces1 = app.get(image1)
if faces1:
  print("face image detected")
else:
  print("face image not detected")

face_image2 = face_align.norm_crop(image1, landmark=faces1[0].kps, image_size=336)

image1 = cv2.imread(imgs_2[0])
faces1 = app.get(image1)
if faces1:
  print("face image detected")
else:
  print("face image not detected")

face_image1 = face_align.norm_crop(image1, landmark=faces1[0].kps, image_size=336)

In [ ]:
from IPython.display import display
from ipywidgets import HBox, Image
import random

def display_images(images, format="png"):
  if format=="png" or True:
    image_widgets = [Image(value=img._repr_png_()) for img in images]
  # Display the images in a row
  display(HBox(image_widgets))

prompt = ""
prompt += "20 years old girl, skinny and bonny, full body, kneeling on bed, serious face, mouth closed, wearing a skirt"
extra_prompt = ""

negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime), text, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, nfsw, extra legs, fused fingers, too many fingers, long neck"
negative_prompt += ""

width = 512
height = 512
num_inference_steps = 50
guidance_scale = 2
num_samples = 3
random_seed = 4574673546
use_hardcoded_seed = False
v2=True
s_scale = 0.1

x_plot_generation = False
y_plot_generation = False
x_plot_parameter = 's_scale'
x_plot_values = [0.1,0.3,0.6,1.0,1.5,2.0]
y_plot_parameter = 'num_inference_steps'
y_plot_values = [30,40,50]
xy_different_seeds = False
face_image = face_image


ip_model = IPAdapterFaceIDPlus(pipe, image_encoder_path, ip_ckpt, device)

if not use_hardcoded_seed:
  random_seed = random.randint(1, 9999999)

if x_plot_generation:
  num_samples=1
  if y_plot_generation:
    images_array = []
    for h in range(len(y_plot_values)):
      images = []
      if y_plot_parameter in globals():
        globals()[y_plot_parameter] = y_plot_values[h]
        if y_plot_parameter != "face_image" and y_plot_parameter != "faceid_embeds":
          print(y_plot_parameter,"set to:",y_plot_values[h])
      for i in range(len(x_plot_values)):
        if x_plot_parameter in globals():
          globals()[x_plot_parameter] = x_plot_values[i]
          if x_plot_parameter != "face_image" and x_plot_parameter != "faceid_embeds":
            print(x_plot_parameter,"set to:",x_plot_values[i])
        if xy_different_seeds and i!=0:
          random_seed = random.randint(1, 9999999)
        print("Seed: ", random_seed)
        generated_image = ip_model.generate(
      prompt=prompt+", "+extra_prompt, negative_prompt=negative_prompt, face_image=face_image, faceid_embeds=faceid_embeds_mean, num_samples=num_samples, width=width, height=height, num_inference_steps=num_inference_steps, seed=random_seed,
      guidance_scale=guidance_scale, shortcut=True, s_scale=s_scale,
  )[0]
        images.append(generated_image)
      images_array.append(images)
    for img in images_array:
      display_images(img)
  elif x_plot_generation:
    images = []
    for i in range(len(x_plot_values)):
      if x_plot_parameter in globals():
        globals()[x_plot_parameter] = x_plot_values[i]
        if x_plot_parameter != "face_image" and x_plot_parameter != "faceid_embeds":
          print(x_plot_parameter,"set to:",x_plot_values[i])
      if xy_different_seeds and i!=0:
        random_seed = random.randint(1, 9999999)
      print("Seed: ", random_seed)
      images.extend(ip_model.generate(
      prompt=prompt+", "+extra_prompt, negative_prompt=negative_prompt, face_image=face_image, faceid_embeds=faceid_embeds_mean, num_samples=num_samples, width=width, height=height, num_inference_steps=num_inference_steps, seed=random_seed,
      guidance_scale=guidance_scale, shortcut=True, s_scale=s_scale,
  ))
    display_images(images)


else:

  print("Seed: ", random_seed)
  images = ip_model.generate(
      prompt=prompt+", "+extra_prompt, negative_prompt=negative_prompt, face_image=face_image, faceid_embeds=faceid_embeds_mean, num_samples=num_samples, width=width, height=height, num_inference_steps=num_inference_steps, seed=random_seed,
      guidance_scale=guidance_scale, shortcut=True, s_scale=s_scale,
  )

  display_images(images)